# Finding frequent itemsets and association rules

### Library: mlxtend
We are going to use the library mlxtend which contains an implementation of the apriori algorithm. Click on the following <a href="http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/association_rules/">link</a>. We recommend to install it using

pip install mlxtend

or

conda install mlxtend

Sometimes using the previous commands the library is not correctly installed, especially in the case when there are multiple Python versions installed on your Jupyter notebook. In that case, we recommend to use the command shown in the next cell (directly in your Jupyter notebook):

In [1]:
import sys
!{sys.executable} -m pip install mlxtend

### One-hot encoding

Sometimes it is required to transform categorical data where features might take more than two values into one-hot encoding, where each feature can take either 0 or 1 (or True/False). For example, let's consider the following set of baskets with each basket containing all the items bought during a simple trip to the supermarket. Using the following script (which uses a TransactionEncoder from mlxtend) we obtain a one-hot encoding of the data, which is required by the apriori method of mlxtend:

In [2]:
from mlxtend.preprocessing import TransactionEncoder

dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
te_ary

array([[False, False, False,  True, False,  True,  True,  True,  True,
        False,  True],
       [False, False,  True,  True, False,  True, False,  True,  True,
        False,  True],
       [ True, False, False,  True, False,  True,  True, False, False,
        False, False],
       [False,  True, False, False, False,  True,  True, False, False,
         True,  True],
       [False,  True, False,  True,  True,  True, False, False,  True,
        False, False]])

Observe that we have now a number of features being equal to the number of items in the data. Each column corresponds to a feature, while each row-column entry specifies whether the corresponding basket contains the corresponding feature or not (e.g. "Milk").

### Frequent itemsets and association rules

Finally we are able to run apriori on the data represented with one-hot encoding using the corresponding methods from mlxtend. Please find below the full code:

In [3]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori


dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)

frequent_itemsets

from mlxtend.frequent_patterns import association_rules

a=association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

a[["antecedents","consequents","support","confidence"]]

,antecedents,consequents,support,confidence
0,(Kidney Beans),(Eggs),0.8,0.80
1,(Eggs),(Kidney Beans),0.8,1.00
2,(Onion),(Eggs),0.6,1.00
3,(Eggs),(Onion),0.6,0.75
4,(Milk),(Kidney Beans),0.6,1.00
5,(Onion),(Kidney Beans),0.6,1.00
6,(Yogurt),(Kidney Beans),0.6,1.00
7,"(Onion, Eggs)",(Kidney Beans),0.6,1.00
8,"(Onion, Kidney Beans)",(Eggs),0.6,1.00
9,"(Kidney Beans, Eggs)",(Onion),0.6,0.75


### DataFrame

One convenient way to load data from an input file and perform some preprocessing on it is to use panda DataFrame. A DataFrame is a "two-dimensional, size-mutable, potentially heterogeneous tabular data." It contains several tools to process and handle tabular data. Click on this <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html">link</a> for a documentation. 

To use a DataFrame we need to install the "pandas" library with any of the methods we saw up to now. 



In [4]:
import pandas as pd

#read order_data.csv and create a DataFrame with that content
data = pd.read_csv(r"order_data.csv",delimiter=" ",header=None)
data

FileNotFoundError: [Errno 2] No such file or directory: 'order_data.csv'

In [ ]:
#we run apriori on the order_data.csv file

import math
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


data = pd.read_csv(r"order_data.csv",delimiter=" ",header=None)

#preprocessing: change to one hot encoding so as to be able to use apriori from mlxtend
d=data.values.tolist()

#removing nan values
for i in range(len(d)):
    j=0
    while(True):
        if (type(d[i][j])==float and math.isnan(d[i][j])) :
            del d[i][j]
            j-=1
        j+=1
        if (j>len(d[i])-1):
            break

te = TransactionEncoder()
te_ary = te.fit(d).transform(d)
df = pd.DataFrame(te_ary, columns=te.columns_)

#computing frequent itemsets and association rules
frequent_itemsets = apriori(df, min_support=0.2, use_colnames=True)

a=association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

#visualizing association rules results
a[["antecedents","consequents","support","confidence"]]

In [ ]:
type(a["antecedents"][99]) #prints frozenset

In [ ]:
#frozenset is very similar to a set in Python with the main difference that cannot be changed.
#so we can check if a rule contains shampoo and honey as follows
i=99
if ("shampoo" in a["antecedents"][i] and "honey" in a["consequents"][i] ):
    print ("The "+str(i)+" rule talks about shampoo and honey:" )
else:
    print ("The "+str(i)+"th rule does not talk about shampoo and honey:" )

In [ ]:
import math
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


data = pd.read_csv(r"mammographic_masses.csv",delimiter=",",header=0)

#preprocessing: change to one hot encoding so as to be able to use apriori from mlxtend
d=data.values.tolist()

#removing nan values
for i in range(0,len(d)):
    j=0
    while(True):
        if (type(d[i][j])==float and math.isnan(d[i][j])) :
            del d[i][j]
            j-=1
        j+=1
        if (j>len(d[i])-1):
            break

#adding attributes
for i in range(len(d)):
    for j in range (len(d[i])):
        d[i][j]=data.columns[j] + "=" +str(d[i][j])

            
te = TransactionEncoder()
te_ary = te.fit(d).transform(d)

df = pd.DataFrame(te_ary, columns=te.columns_)

#this part needs to be filled...